In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from tabulate import tabulate
import sys
import copy
sys.path.append('C:\\Users\\chery\\Documents\\SeattlePolice911Response')
from utils.baseML import BaseML
from datetime import datetime
from sklearn import preprocessing

In [ ]:
spir = BaseML("Processed SPIR Data/SPIR_encoded_2021116.csv")

In [ ]:
spir.df.columns

In [ ]:
spir_process_remove_cat= spir.df.drop(['Unnamed: 0', "Event Clearance Description", "Event Clearance SubGroup", "Event Clearance Group", 
"Initial Type Subgroup", "Initial Type Group",
'Initial Type Description', 'at_scene_time', 'event_clear_time', 'time_at_scene', 'Census Tract'], axis=1)

In [ ]:
from scipy import stats
spir_process_for_outliers=spir_process_remove_cat[spir_process_remove_cat.time_at_scene_seconds > 0]
#spir_process_for_outliers= spir_process_for_outliers[(np.abs(stats.zscore(spir_process_for_outliers, axis=0)) < 3).all(axis=1)]
spir_process_only_outliers = spir_process_remove_cat[(np.abs(stats.zscore(spir_process_remove_cat, axis=0)) >= 3).any(axis=1)]
#spir_process_only_outliers = spir_process_remove_cat[spir_process_remove_cat.time_at_scene_seconds < 0]
zscore = stats.zscore(spir_process_remove_cat, axis=1)

In [ ]:
print(len(spir_process_only_outliers))
print(len(spir_process_for_outliers))

In [ ]:
spir_process_outliers = spir.df.drop(['Unnamed: 0', "Event Clearance Description", "Event Clearance SubGroup", "Event Clearance Group", 
"Initial Type Subgroup", "Initial Type Group",
'Initial Type Description', 'at_scene_time', 'event_clear_time', 'time_at_scene',], axis=1)
spir_process_outliers = spir_process_outliers.iloc[spir_process_only_outliers.index]
spir_process_outliers.to_csv("Processed SPIR Data/SPIR_outliers_20211207.csv")

In [ ]:
spir_process = spir.df.drop(['Unnamed: 0', "Event Clearance Description", "Event Clearance SubGroup", "Event Clearance Group", 
"Initial Type Subgroup", "Initial Type Group",
'Initial Type Description', 'at_scene_time', 'event_clear_time', 'time_at_scene',], axis=1)
len(spir_process)

In [ ]:
spir_process_for_outliers.index

In [ ]:
spir_process = spir_process.iloc[spir_process_for_outliers.index]
len(spir_process)

from scipy import stats
spir_process= spir_process[(np.abs(stats.zscore(spir_process)) < 3).all(axis=1)]

In [ ]:
spir_process.to_csv("Processed SPIR Data/SPIR_no_outliers_removed_20211207.csv")

In [ ]:
spir_process=spir_process[spir_process.time_at_scene_seconds > 0]

In [ ]:
print(len(spir_process))

In [ ]:
spir_baseML = BaseML()
spir_baseML.df = spir_process

In [ ]:
spir_process_not_normalized = spir.df.iloc[spir_process_for_outliers.index]

In [ ]:
spir_process = spir_baseML.normalize_features()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(spir_process)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
x_pca=pca.fit_transform(spir_process)

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer, InterclusterDistance, silhouette_visualizer
clf = KMeans()


In [ ]:

visualizer = KElbowVisualizer(clf, k=(4,12))

visualizer.fit(x_pca)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
silhouette_visualizer(KMeans(6, random_state=42), x_pca, colors='yellowbrick')

In [ ]:
clf = KMeans(n_clusters=7)
y_pred_7 = clf.fit_predict(x_pca)

In [ ]:
plt.scatter(x_pca[:,0], x_pca[:,1], c=y_pred_7)
plt.show()

In [ ]:
cluster_labels = clf.labels_
cl_label_df = pd.DataFrame(cluster_labels, columns = ["cluster_label"])

In [ ]:
len(cluster_labels)
print(cl_label_df.head)

In [ ]:
spir_process_not_normalized.reset_index(drop=True, inplace=True)

In [ ]:
from datetime import date
spir_with_labels = pd.concat([spir_process_not_normalized, cl_label_df], axis=1)
spir_with_labels.to_csv(f"Processed SPIR Data/SPIR_with_labels_{date.today()}_2.csv")

In [ ]:
print(len(spir.df))

In [ ]:
spir_with_labels.head

In [ ]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool

In [ ]:
filename = "plot_bokeh_20211124.html"
palette =['aqua', 'aquamarine', 'azure', 'black', 'blue', 
        'brown', 'chartreuse', 'coral', 'crimson', 'cyan', 
        'darkblue', 'darkgreen', 'fuschsia', 'gold', 'indigo', 
        'lavender', 'lime', 'magenta', 'olive', 'orange', 
        'orangered', 'orchid', 'pink', 'red','salmon']#['red','green','blue','yellow']
colors =[]

for i in range(spir_with_labels.shape[0]):
    colors.append(palette[cluster_labels[i]])

In [ ]:
#cannot use seaborn palette for bokeh

#plot with boken
output_file(filename)
source = ColumnDataSource(
        data=dict(x=x_pca[:,0],y=x_pca[:,1],
            #event = spir_with_labels['event'],
            time = spir_with_labels['time_at_scene_seconds'], 
            census_tract = spir_with_labels["Census Tract"],
            year = spir_with_labels["at_scene_time_year"],
            percent_white = spir_with_labels["white"],
            income = spir_with_labels["income"],
            college = spir_with_labels["bachelors"],
            colors=colors), )

#print(colors)
hover = HoverTool(tooltips=[
            #("event", "@event"),
            ("time", "@time"),
            ("census_tract", "@census_tract"),
            ("year", "@year"),
            ("percent_white", "@percent_white"),
            ("income", "@income"),
            ("college", "@college")])

p = figure(plot_width=1000, plot_height=1000, tools=[hover],
            title="spir clustering")

p.circle(x='x',y= 'y', size=10, source=source,fill_color="colors")

show(p)